# News Sentiment Analysis

## Import Libraries

In [1]:
# // TODO: TINGS
#     ✓ Find API for data collection
#     - GPU Selector
#     - Data Gathering
#     - Data Cleaning
#     - Feature generation
#     - Feature Engineering/selection
#     - Model Train
#     - Model Test
#     - Model Evaluation

In [ ]:
# Common Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

# Cleaner output
from tqdm import tqdm

# Deep Learning Libraries
import torch

# Add the path to the API Scraper
## Project Path
project_path = "../"

## Add the path to API Scraper
sys.path.append(os.path.abspath(os.path.join(project_path, "lib")))

# Custom API Scraping Libraries
from scraper import get_cached_news_metadata, extract_text_from_url

## Fetch Data

In [3]:
def scrape_data(n_pages: int = 10, before_date: str = "2025-12", path: str = ".") -> pd.DataFrame:
    
    articles = []

    for i in tqdm(range(n_pages), desc="Fetching News Data...", unit="news"):
        metadata = get_cached_news_metadata(page=i, before_date=before_date, path=path)

        data_list = metadata.get("data", [])

        for article in data_list:
            # extract sentiment (first entity if exists)
            entities = article.get("entities", [])
            if entities and "sentiment_score" in entities[0]:
                sentiment = entities[0]["sentiment_score"]
            else:
                sentiment = None

            # store sentiment as its own field inside article
            article["sentiment"] = sentiment

            articles.append(article)

    # final dataframe
    return pd.DataFrame(articles)

### Caching

In [4]:
# caching the dataset
before_date = "2025-12"

data_path = os.path.join(project_path,f"news_cache/{before_date}/csv/")
os.makedirs(data_path, exist_ok=True) # if the directory exist no need to make

cached_file = os.path.join(data_path, f"{before_date}_news_data.csv")

get_new_data = False

# We will cache the data so that it will load faster
if os.path.exists(cached_file) and not get_new_data:
    print("Loading cached dataset...")
    news_df = pd.read_csv(cached_file)
    print("Cached dataset loaded")

elif os.path.exists(cached_file) and get_new_data:
    print("Overwriting old data and caching new data...")
    news_df = scrape_data(n_pages=95, before_date= before_date, path= project_path)
    news_df.to_csv(cached_file, index=False)
    print("Done Overwriting old data and caching new data...")

else:
    print("Creating and caching dataset...")
    news_df = scrape_data(n_pages=95, before_date= before_date, path= project_path)
    news_df.to_csv(cached_file, index=False)
    print("Finished Caching")


Loading cached dataset...
Cached dataset loaded


### Fetch the Text from URL

In [20]:
news_df["text"] = news_df["url"].apply(extract_text_from_url)

ArticleException: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.manilatimes.net/2025/11/30/tmt-newswire/globenewswire/airlife-gases-acquires-control-of-royal-helium-ltd/2233736 on URL https://www.manilatimes.net/2025/11/30/tmt-newswire/globenewswire/airlife-gases-acquires-control-of-royal-helium-ltd/2233736

## EDA

### View the Data

In [5]:
news_df.head()

,uuid,title,description,keywords,snippet,url,image_url,language,published_at,source,relevance_score,entities,similar,sentiment
0,487e6a88-d3c2-4ae1-8dc2-26af6b31d688,2025: The Year Of Alphabet (GOOG),No stock has seen a bigger jump recently than ...,NaN,vzphotos/iStock Editorial via Getty Images\n\n...,https://seekingalpha.com/article/4848680-2025-...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:30:00.000000Z,seekingalpha.com,NaN,"[{'symbol': 'GOOGL', 'name': 'Alphabet Inc.', ...",[],0.0000
1,92b5c2bd-d324-4ae8-b115-2cfd95a8fa98,Why I'm Doubling Down On My Adobe Position (NA...,"Adobe's revenue is highly predictable, driven ...",NaN,To say that Adobe ( ADBE ) stock has not had a...,https://seekingalpha.com/article/4848762-why-i...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:25:01.000000Z,seekingalpha.com,NaN,"[{'symbol': 'ADBE', 'name': 'Adobe Inc.', 'exc...",[],0.0000
2,9084e5f1-75f5-4f15-aa3d-0676073b4aaf,Global week ahead: The start of a Santa Rally ...,NaN,"STOXX 600, business news",And just like that... December is upon us. It'...,https://www.cnbc.com/2025/11/30/global-week-ah...,https://image.cnbcfm.com/api/v1/image/10823257...,en,2025-11-30T05:10:58.000000Z,cnbc.com,NaN,"[{'symbol': 'M', 'name': ""Macy's, Inc."", 'exch...",[],0.6908
3,487e6a88-d3c2-4ae1-8dc2-26af6b31d688,2025: The Year Of Alphabet (GOOG),No stock has seen a bigger jump recently than ...,NaN,vzphotos/iStock Editorial via Getty Images\n\n...,https://seekingalpha.com/article/4848680-2025-...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:30:00.000000Z,seekingalpha.com,NaN,"[{'symbol': 'GOOGL', 'name': 'Alphabet Inc.', ...",[],0.0000
4,92b5c2bd-d324-4ae8-b115-2cfd95a8fa98,Why I'm Doubling Down On My Adobe Position (NA...,"Adobe's revenue is highly predictable, driven ...",NaN,To say that Adobe ( ADBE ) stock has not had a...,https://seekingalpha.com/article/4848762-why-i...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:25:01.000000Z,seekingalpha.com,NaN,"[{'symbol': 'ADBE', 'name': 'Adobe Inc.', 'exc...",[],0.0000


### Check for Missing Values

In [6]:
is_na = pd.DataFrame(news_df.isna().sum())
is_na.columns = ["Number_Missing"]
is_na["Missing_Percentage"] = (is_na["Number_Missing"] / len(news_df) * 100)
print(is_na)

                 Number_Missing  Missing_Percentage
uuid                          0            0.000000
title                         0            0.000000
description                   4            1.403509
keywords                    131           45.964912
snippet                       0            0.000000
url                           0            0.000000
image_url                     0            0.000000
language                      0            0.000000
published_at                  0            0.000000
source                        0            0.000000
relevance_score             285          100.000000
entities                      0            0.000000
similar                       0            0.000000
sentiment                     0            0.000000


## Data Cleaning

In [ ]:
def clean_text(text: str, language: str='english', tokenize: bool = False, remove_stop_words: bool = False, stem_words: bool = False, remove_url: bool = False, remove_emojis: str = "convert", expand_abbreviations: bool=False):
    """
    #### Description:
    This function is to clean the text from stopwords, punctuation and return a clean text for further analysis

    Args:
        text (str):
            The dataframe containing the text data
        
        language (str):
            This are the available languages for the stopwords:
            - "catalan"
            - "czech"
            - "german"
            - "greek"
            - "english"
            - "spanish"
            - "finnish"
            - "french"
            - "hungarian"
            - "icelandic"
            - "italian"
            - "latvian"
            - "dutch"
            - "polish"
            - "portuguese"
            - "romanian"
            - "russian"
            - "slovak"
            - "slovenian"
            - "swedish"
            - "tamil"
        
        tokenize (bool):
            True = return tokenized data
            False = return untokenized data
        
        remove_stop_words (bool):
            True = remove stop words
            False = do not remove stop words

        stem_words (bool):
            True = get the base words (i.e. spraying -> spray)
            False = leave the words as is

        remove_url (bool):
            True = Remove the url in the text
            False = leave the text as is
        
        remove_emojis (str):
            "remove" = Removes the emoji in text
            "convert = converts emoji to text (e.g. ❤️ -> :red_heart:)
            "keep" = keeps the emoji as is
        
        expand_abbreviations (bool):
            True = Expand abbreviations (e.g. brb -> "be right back")
            False = Keep the abbriviations as is
    """

    # slang dictionary
    slang_dict = {
        "brb": "be right back",
        "afk": "away from keyboard",
        "gg": "good game",
        "ggwp": "good game well played",
        "lol": "laughing out loud",
        "idk": "I do not know",
        "imo": "in my opinion",
        "lmao": "laughing my ass off",
        "lmfao": "laughing fucking my ass off",
        "sus": "suspicious",
        "rekt": "wrecked",
        "noob": "new player",
        "af": "as hell",
        "wtf": "what the fuck",
        "wth": "what the heck",
        "omg": "oh my god",
        "ty": "thank you",
        "plz": "please",
        "pls": "please",
        "u": "you",
        "r": "are",
        "thx": "thanks",
        "fr": "for real",
        "til": "today i learned",
        "asap": "as soon as possible",
        "g2g": "got to go",
        "gtg": "got to go",
        "nc": "nice",
        "fyi": "for your information",
        "ttyl": "talk to you later",
        "fb": "facebook",
        "msg": "message",
        "hifw": "how i feel when",
        "tfw": "the feeling when",
        "mfw": "my face when",
        "mrw": "my reaction when",
        "ifyp": "i feel your pain",
        "tntl": "trying not to laugh",
        "jk": "just kidding",
        "idc": "i dont care",
        "ily": "i love you",
        "imu": "i miss you",
        "zzz": "sleeping, bored, tired",
        "ftw": "for the win",
        "tbh": "to be honest",
        "ftl": "for the loss",
        "smh": "shaking my head",
        "srsly": "seriously",
        "afaik": "as far as i know",
        "dm": "direct message",
        "tldr": "too long didnt read",
        "irl": "in real life",
        "gl": "goodluck",
        "ruok": "are you ok",
        "w": "win"
    }

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words(language))

    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]
    
    def remove_special_characters(text):
        # keep letters, numbers, underscores, colons (for demojized emojis)
        text = re.sub('[^a-zA-Z0-9_]', ' ', text)
        text = re.sub('\s+', ' ', text)
        return text

    def stem_text(tokens):
        return [stemmer.stem(t) for t in tokens]

    def remove_stopwords_func(tokens):
        return [w for w in tokens if w not in stop_words]

    def remove_url_func(text):
        return re.sub(r'https?://\S+|www\.\S+', '', text)

    def expand_slang(text):
        words = text.split()
        return " ".join([slang_dict.get(w.lower(), w) for w in words])

    # Clean process
    text = contractions.fix(text)                        # fixing contraction

    text = text.strip().lower()                          # lowercase + trim

    if remove_url:
        text = remove_url_func(text)                     # remove url
    
    # Handle emojis
    if remove_emojis.lower() == "remove":
        text = remove_special_characters(text)  # removes emojis

    elif remove_emojis.lower() == "convert":
        text = emoji.demojize(text, language="en")  # e.g.,  -> ❤️ -> :red_heart:

    elif remove_emojis.lower() == "keep":
        pass

    if expand_abbreviations:
        text = remove_special_characters(text)   # <--- clean before slang expansion
        text = expand_slang(text)

    text = remove_special_characters(text)               # Remove other special characters (but preserve converted emojis with underscores)
    
    tokens = tokenize_text(text)                         # tokenize words

    if remove_stop_words:
        tokens = remove_stopwords_func(tokens)           # remove stopwords
        
    if stem_words:
        tokens = stem_text(tokens)                       # stemming

    if tokenize:
        return tokens                                    # return as tokens
    else:
        return " ".join(tokens)                          # return as string